## Nhóm 7
## 1512108 - Võ Tiến Đạt
## 1512030 - Trần Quốc Bảo

Dự đoán: nhiệt độ cao nhất của ngày hôm sau<br>
Lợi ích: mọi hoạt động diễn ra ngoài trời đều bị ảnh hưởng bởi thời tiết, nhất là sản xuất nông nghiệp. Vậy nếu dự đoán trước được thì ta có biện pháp phòng tránh, ứng biến.

Dữ liệu thời tiết được lấy trên trang https://DarkSky.net<br>
Về cách lấy dữ liệu trang này cung cấp 1000 requests một ngày đối với mỗi tài khoản.<br>
Dữ liệu lấy với địa điểm ở thành phố Hồ Chí Minh từ năm 1998-2017.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin, clone
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from copy import deepcopy

In [2]:
data_df = pd.read_csv('weatherdata.csv')

In [3]:
data_df.head()

,time,summary,sunriseTime,sunsetTime,moonPhase,precipType,temperatureHigh,temperatureHighTime,temperatureLow,temperatureLowTime,...,uvIndexTime,visibility,temperatureMin,temperatureMinTime,temperatureMax,temperatureMaxTime,apparentTemperatureMin,apparentTemperatureMinTime,apparentTemperatureMax,apparentTemperatureMaxTime
0,883587600,Partly cloudy throughout the day.,883609978,883651381,0.09,rain,33.03,883638000,21.02,883695600,...,883630800,NaN,22.02,883609200,33.03,883638000,22.59,883609200,34.64,883638000
1,883674000,Partly cloudy throughout the day.,883696401,883737813,0.12,rain,33.03,883724400,21.02,883778400,...,883713600,NaN,21.02,883695600,33.03,883724400,21.63,883695600,34.64,883724400
2,883760400,Partly cloudy throughout the day.,883782824,883824245,0.16,rain,33.03,883814400,23.71,883872000,...,883800000,NaN,21.02,883778400,33.03,883814400,21.63,883778400,34.91,883818000
3,883846800,Partly cloudy throughout the day.,883869245,883910677,0.20,rain,32.32,883904400,23.71,883944000,...,883886400,NaN,23.71,883872000,32.32,883904400,24.30,883857600,35.83,883890000
4,883933200,Humid and partly cloudy throughout the day.,883955666,883997109,0.23,rain,33.51,883980000,22.82,884034000,...,883972800,NaN,23.71,883944000,33.51,883980000,24.29,883944000,37.00,883980000


In [4]:
data_df.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7305 entries, 0 to 7304
Data columns (total 33 columns):
time                           7305 non-null int64
summary                        7305 non-null object
sunriseTime                    7305 non-null int64
sunsetTime                     7305 non-null int64
moonPhase                      7305 non-null float64
precipType                     5144 non-null object
temperatureHigh                7305 non-null float64
temperatureHighTime            7305 non-null int64
temperatureLow                 7305 non-null float64
temperatureLowTime             7305 non-null int64
apparentTemperatureHigh        7305 non-null float64
apparentTemperatureHighTime    7305 non-null int64
apparentTemperatureLow         7305 non-null float64
apparentTemperatureLowTime     7305 non-null int64
dewPoint                       7305 non-null float64
humidity                       7305 non-null float64
pressure                       7305 non-null float64
windSpee

In [5]:
def convert_localhour(df,at):
    df[at] = pd.to_datetime(df[at],unit='s').dt.tz_localize('UTC').dt.tz_convert('Asia/Ho_Chi_Minh').dt.hour
    
def convert_localday(df,at):
    df[at] = pd.to_datetime(df[at],unit='s').dt.tz_localize('UTC').dt.tz_convert('Asia/Ho_Chi_Minh')

In [6]:
class ColDropper(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, X_df, y=None):
        return self
    def transform(self, X_df, y=None):
        data_df = X_df.copy()
        #chuyển time sang giờ
        convert_localhour(data_df,'sunriseTime')
        convert_localhour(data_df,'sunsetTime')
        convert_localhour(data_df,'uvIndexTime')
        convert_localhour(data_df,'temperatureHighTime')
        convert_localhour(data_df,'temperatureLowTime')
        convert_localhour(data_df,'temperatureMinTime')
        convert_localhour(data_df,'temperatureMaxTime')
        convert_localhour(data_df,'apparentTemperatureMinTime')
        convert_localhour(data_df,'apparentTemperatureMaxTime')
        convert_localhour(data_df,'apparentTemperatureHighTime')
        convert_localhour(data_df,'apparentTemperatureLowTime')

        #loại bỏ các cột không cần
        dropped_cols = ['summary', 'windGustTime', 'windGust','visibility', 'time', 'precipType']
        data_df.drop(dropped_cols, axis=1, inplace=True)
        
        return data_df

In [7]:
dayOfBefore = 1

In [8]:
#tách X y
y_sr = data_df["apparentTemperatureMax"][dayOfBefore:]
X_df = data_df.drop("apparentTemperatureMax", axis=1)[:-dayOfBefore]

In [9]:
#chia theo tỉ lệ train:validation:test là 60%:20%:20%
train_X_df, val_test_X_df, train_y_sr, val_test_y_sr = train_test_split(X_df, y_sr, test_size=0.4, random_state=40)
val_X_df, test_X_df, val_y_sr, test_y_sr = train_test_split(val_test_X_df, val_test_y_sr, test_size=0.5, random_state=40)

train_y = train_y_sr.values
val_y = val_y_sr.values
test_y = test_y_sr.values

In [10]:
num_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy = 'mean'))])

full_preprocess_pipeline = Pipeline(steps=[('coldrop', ColDropper()),
                                         ('num', num_transformer)])

In [11]:
preprocessed_train_X = full_preprocess_pipeline.fit_transform(train_X_df)
preprocessed_val_X = full_preprocess_pipeline.transform(val_X_df)

In [12]:
alpha = [0.0001, 0.001, 0.01, 0.1, 1, 10]
best_mlp_regr = None
best_val_score = -float('inf')
for al in alpha:
    regr = MLPRegressor(hidden_layer_sizes=(100,), alpha=al)
    regr.fit(preprocessed_train_X, train_y)
    train_y_pred = regr.predict(preprocessed_train_X)
    val_y_pred = regr.predict(preprocessed_val_X)
    score_train = r2_score(train_y, train_y_pred)
    score_val = r2_score(val_y, val_y_pred)
    if best_val_score<score_val:
        best_val_score = score_val
        best_mlp_regr = deepcopy(regr)
    print('alpha = {0}, score_train = {1:0.2f}, score_val = {2:0.2f}'.format(al,score_train, score_val))

alpha = 0.0001, score_train = 0.59, score_val = 0.62
alpha = 0.001, score_train = 0.62, score_val = 0.65
alpha = 0.01, score_train = 0.65, score_val = 0.67
alpha = 0.1, score_train = 0.65, score_val = 0.68
alpha = 1, score_train = 0.65, score_val = 0.67
alpha = 10, score_train = 0.67, score_val = 0.70


In [13]:
regr = linear_model.LinearRegression()
regr.fit(preprocessed_train_X, train_y)
train_y_pred = regr.predict(preprocessed_train_X)
val_y_pred = regr.predict(preprocessed_val_X)
score_train = r2_score(train_y, train_y_pred)
score_val = r2_score(val_y, val_y_pred)
print('score_train = {0:0.2f}, score_val = {1:0.2f}'.format(score_train, score_val))

score_train = 0.68, score_val = 0.70


In [14]:
preprocessed_test_X = full_preprocess_pipeline.transform(test_X_df)

In [15]:
# test_y_pred = best_mlp_regr.predict(preprocessed_test_X)
# print('MLP Regression')
# print("Mean squared error: %.2f" % mean_squared_error(test_y, test_y_pred))
# print('Variance score: %.2f' % r2_score(test_y, test_y_pred), end='\n\n')

In [16]:
test_y_pred = regr.predict(preprocessed_test_X)
print("Linear Regression")
print("Mean squared error: %.2f" % mean_squared_error(test_y, test_y_pred))
print('Variance score: %.2f' % r2_score(test_y, test_y_pred))

Linear Regression
Mean squared error: 3.40
Variance score: 0.69
